In [ ]:
# //a[@class="JtKRv"] - XPath for A tag with link and text
# //time[@class="hvbAAd"] - XPath for Datetime

In [ ]:
# company_name = "Apple"
# site_name = "bbc"
# num_days_ago = "1"

# # URL to scrape
# url = f'https://news.google.com/search?q={company_name}%20site%3A{site_name}.com%20when%3A{num_days_ago}d&hl=en-GB&gl=GB&ceid=GB%3Aen'
# driver.get(url)

# # Find elements by xpaths
# article_elements = driver.find_elements(By.XPATH, '//a[@class="JtKRv"]')
# datetime_elements = driver.find_elements(By.XPATH, '//time[@class="hvbAAd"]')

# # Data lists
# data = []

# # Check if the number of articles and datetimes are the same
# if len(article_elements) == len(datetime_elements):
#     for article, datetime_element in zip(article_elements, datetime_elements):
#         # Extracting the datetime string and parsing it
#         datetime_str = datetime_element.get_attribute('datetime')
#         parsed_datetime = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%SZ")
#         date = parsed_datetime.date()
#         time = parsed_datetime.time()
        
#         # Extract article name and link
#         article_name = article.text
#         article_link = article.get_attribute('href')

#         # Append to data list
#         data.append([date, time, article_name, article_link, site_name])

# # Create a DataFrame
# columns = ['Date', 'Time', 'Article Name', 'Article Link', 'Site Name']
# df = pd.DataFrame(data, columns=columns)

# # Close the driver
# driver.quit()

# Output DataFrame
# df

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

# Setup the Chrome WebDriver
chromedriver_path = ChromeDriverManager().install()
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

company_name = "Apple"
site_names = ["bbc", "wsj", "nytimes", "forbes", "finance.yahoo", "bloomberg", "ft"]
num_ago = "1"
time_ago = "y"  # for year or "d" for days

# Data lists
data = []

# Loop over each site
for site_name in site_names:
    # URL to scrape
    url = f'https://news.google.com/search?q={company_name}%20site%3A{site_name}.com%20when%3A{num_ago}{time_ago}&hl=en-GB&gl=GB&ceid=GB%3Aen'
    driver.get(url)

    # Scroll to the bottom to load all articles
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        driver.implicitly_wait(10)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Find elements by xpaths
    article_elements = driver.find_elements(By.XPATH, '//a[@class="JtKRv"]')
    datetime_elements = driver.find_elements(By.XPATH, '//time[@class="hvbAAd"]')

    # Check if the number of articles and datetimes are the same
    if len(article_elements) == len(datetime_elements):
        for article, datetime_element in zip(article_elements, datetime_elements):
            article_name = article.text

            # Check if the article title contains the company name
            if company_name.lower() in article_name.lower():
                # Extracting the datetime string and parsing it
                datetime_str = datetime_element.get_attribute('datetime')
                parsed_datetime = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%SZ")
                date = parsed_datetime.date()
                time = parsed_datetime.time()

                # Extract article link
                article_link = article.get_attribute('href')

                # Append to data list
                data.append([date, time, article_name, article_link, site_name])

# Create a DataFrame
columns = ['Date', 'Time', 'Article Name', 'Article Link', 'Site Name']
df = pd.DataFrame(data, columns=columns)

# Sort and remove duplicates
df = df.sort_values(by=["Date", "Site Name"])
df = df.drop_duplicates()

# Close the driver
driver.quit()

# Output DataFrame
df

,Date,Time,Article Name,Article Link,Site Name
102,2023-06-30,07:00:00,Goldman Is Looking for a Way Out of Its Partne...,https://news.google.com/articles/CBMiZGh0dHBzO...,wsj
39,2023-07-01,07:00:00,An apple caramel pecan pie born from a railway...,https://news.google.com/articles/CBMiaGh0dHBzO...,bbc
83,2023-07-19,07:00:00,Apple’s Next Killer App Is…Voicemail?,https://news.google.com/articles/CBMiS2h0dHBzO...,wsj
30,2023-07-20,07:00:00,Apple slams UK surveillance-bill proposals,https://news.google.com/articles/CBMiLGh0dHBzO...,bbc
118,2023-08-11,07:00:00,Amazon Wants You to Pay With Your Palm. It's a...,https://news.google.com/articles/CBMicmh0dHBzO...,wsj
...,...,...,...,...,...
265,2024-06-22,09:30:00,Chinese consumers love AI smartphones but Appl...,https://news.google.com/articles/CBMiVmh0dHBzO...,finance.yahoo
266,2024-06-22,15:50:02,Is Apple Stock Going to $240? 1 Wall Street An...,https://news.google.com/articles/CBMiRWh0dHBzO...,finance.yahoo
159,2024-06-22,17:08:00,"Apple Loop: iPhone 16 Design Leaks, New M4 Mac...",https://news.google.com/articles/CBMigAFodHRwc...,forbes
41,2024-06-22,10:00:00,"What's News in Markets: Nvidia Effect, S&P Mil...",https://news.google.com/articles/CBMijQFodHRwc...,wsj


In [2]:
df.to_csv("News Data.csv")

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from pymongo import MongoClient

# Setup MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client["news_articles_db"]

# List of companies and sites
companies = ["Apple", "Microsoft", "Google", "Amazon", "Nvidia", "Meta", "Tesla"]
site_names = ["bbc", "wsj", "nytimes", "forbes", "finance.yahoo", "bloomberg", "ft"]
num_ago = "1"
time_ago = "y" # for year or "d" for days

# Setup the Chrome WebDriver
chromedriver_path = ChromeDriverManager().install()
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

for company_name in companies:
    for site_name in site_names:
        # URL to scrape
        url = f'https://news.google.com/search?q={company_name}%20site%3A{site_name}.com%20when%3A{num_ago}{time_ago}&hl=en-GB&gl=GB&ceid=GB%3Aen'
        driver.get(url)

        # Implement pagination
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            driver.implicitly_wait(10)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Find elements by xpaths
        article_elements = driver.find_elements(By.XPATH, '//a[@class="JtKRv"]')
        datetime_elements = driver.find_elements(By.XPATH, '//time[@class="hvbAAd"]')

        # Data collection
        data = []
        if len(article_elements) == len(datetime_elements):
            for article, datetime_element in zip(article_elements, datetime_elements):
                article_name = article.text

                # Check if the article title contains the company name
                if company_name.lower() in article_name.lower():
                    # Extracting the datetime string and parsing it
                    datetime_str = datetime_element.get_attribute('datetime')
                    if datetime_str.endswith('Z'):
                        datetime_str = datetime_str[:-1]
                    parsed_datetime = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S")
                    date_time = parsed_datetime.strftime("%Y-%m-%d %H:%M:%S")
                    
                    # Extract article link
                    article_link = article.get_attribute('href')
            
                    # Append to data list
                    data.append({
                        "DateTime": date_time,
                        "Article Name": article_name,
                        "Article Link": article_link,
                        "Site Name": site_name
                    })

        # Insert data into MongoDB
        if data:
            collection = db[company_name]
            # Remove duplicates and insert
            keys = {"Article Link": {"$exists": True}}
            collection.create_index([("Article Link", 1)], unique=True)
            for item in data:
                collection.update_one({"Article Link": item["Article Link"]}, {'$setOnInsert': item}, upsert=True)

# Close the driver
driver.quit()